In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
# df = pd.read_csv('./../Naive_Bayes/tweets/training1600000processednoemoticon.csv', names=['target','id','date','flag','user','text'])
# df.head()

In [ ]:
# df['target'].value_counts()

In [2]:
df = pd.read_csv('./../Naive_Bayes/tweets/p_n_n_dataset.csv')
df.head()

,id,date,time,user_id,username,name,tweet,language,label
0,1.340000e+18,12/17/2020,10:56:06,2.760579e+09,thecoolan,Nishat K,@citizenjeff @lauferlaw @donwinslow Haven’t ha...,en,negative
1,1.330000e+18,11/30/2020,6:44:40,1.020000e+18,rmillennielle,#Resistance Millennielle,@tkdylan People have agency. Lin Wood is a her...,en,negative
2,1.330000e+18,11/8/2020,11:30:16,3.605334e+08,frankpeteani,Frank Peteani,@PhoenixAEW I idea that WWE had him coming out...,en,negative
3,1.320000e+18,11/6/2020,5:06:59,9.727358e+07,clarknt67,"🌹 Clark, a proud citizen of GoFundMe nation",Maxwell will probably pee in the house before ...,en,negative
4,1.320000e+18,11/4/2020,10:36:29,1.936187e+09,doxiemomdeb,Henry and Milo’s Hoomom,@msamson56 I’m amazed. HOW can people support ...,en,negative


In [3]:
df['label'].value_counts()

neutral     77028
positive    51994
negative    45622
Name: label, dtype: int64

In [107]:
import google_trans_new
from google_trans_new import google_translator  

  
translator = google_translator()  
translate_text = translator.translate(df.loc[1]['tweet'], lang_src='en', lang_tgt='lv')


print(df.loc[1]['tweet'])
print(translate_text)

google_new_transError: 429 (Too Many Requests) from TTS API. Probable cause: Unknown

In [ ]:
df = df[pd.notnull(df['tweet'])]
df.size

In [ ]:
df.head()

In [87]:
for index, row in df.iterrows():
    row['message'] = translator.translate(row['tweet'], lang_src='en', lang_tgt='lv')
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

google_new_transError: 429 (Too Many Requests) from TTS API. Probable cause: Unknown

In [4]:
label_dict = {'neutral': 0, 'positive': 1, 'negative': 2}
df['label'] = df.label.replace(label_dict) 
df['label'].value_counts()

0    77028
1    51994
2    45622
Name: label, dtype: int64

In [5]:
df['message_eng'] = df.tweet
df.head()

,id,date,time,user_id,username,name,tweet,language,label,message_eng
0,1.340000e+18,12/17/2020,10:56:06,2.760579e+09,thecoolan,Nishat K,@citizenjeff @lauferlaw @donwinslow Haven’t ha...,en,2,@citizenjeff @lauferlaw @donwinslow Haven’t ha...
1,1.330000e+18,11/30/2020,6:44:40,1.020000e+18,rmillennielle,#Resistance Millennielle,@tkdylan People have agency. Lin Wood is a her...,en,2,@tkdylan People have agency. Lin Wood is a her...
2,1.330000e+18,11/8/2020,11:30:16,3.605334e+08,frankpeteani,Frank Peteani,@PhoenixAEW I idea that WWE had him coming out...,en,2,@PhoenixAEW I idea that WWE had him coming out...
3,1.320000e+18,11/6/2020,5:06:59,9.727358e+07,clarknt67,"🌹 Clark, a proud citizen of GoFundMe nation",Maxwell will probably pee in the house before ...,en,2,Maxwell will probably pee in the house before ...
4,1.320000e+18,11/4/2020,10:36:29,1.936187e+09,doxiemomdeb,Henry and Milo’s Hoomom,@msamson56 I’m amazed. HOW can people support ...,en,2,@msamson56 I’m amazed. HOW can people support ...


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['label', 'data_type']).count()

id   date   time  user_id  username   name  tweet  \
label data_type                                                         
0     train      65474  65474  65474    65474     65474  65473  65474   
      val        11554  11554  11554    11554     11554  11554  11554   
1     train      44195  44195  44195    44195     44195  44195  44195   
      val         7799   7799   7799     7799      7799   7799   7799   
2     train      38778  38778  38778    38778     38778  38777  38778   
      val         6844   6844   6844     6844      6844   6843   6844   

                 language  message_eng  
label data_type                         
0     train         65474        65474  
      val           11554        11554  
1     train         44195        44195  
      val            7799         7799  
2     train         38778        38778  
      val            6844         6844

In [7]:
df[df.data_type=='train']['label'].value_counts()

0    65474
1    44195
2    38778
Name: label, dtype: int64

In [8]:
df_val = [df[df.data_type=='val'].message_eng, df[df.data_type=='val'].label]

df_train = [df[df.data_type=='train'].message_eng, df[df.data_type=='train'].label]
df_train = pd.concat(df_train, axis=1, keys=["message_eng", "label"])

df_0 = df_train[df_train['label']==0]
df_1 = df_train[df_train['label']==1]
df_2 = df_train[df_train['label']==2]

df_0_downsampled = df_0.sample(df_2.shape[0])
df_1_downsampled = df_1.sample(df_2.shape[0])

df_train = pd.concat([df_0_downsampled, df_1_downsampled, df_2])

df_train['label'].value_counts()

0    38778
1    38778
2    38778
Name: label, dtype: int64

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].message_eng.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].message_eng.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\transformers\tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [10]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [13]:
model.load_state_dict(torch.load('models4/finetuned_BERT_ENG_epoch_1.model', map_location=device))

<All keys matched successfully>

In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [16]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [17]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict = {0: 0, 1: 1, 2: 2,}
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [18]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [19]:
epoch = 2

In [20]:
while epoch < (epochs+1):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'models4/finetuned_BERT_ENG_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    
    epoch = epoch + 1

Epoch 2:   0%|          | 0/49483 [00:00<?, ?it/s]

KeyboardInterrupt: 